## shock_cooling_curve: A tutorial

### Motivation: 

Early time data from core collapse events contain valuable information about their progenitor system. Some core collapse events produce double peaked light curves: details about the the location of shock breakout (we can think of this as the radius of the extended envelope of the progenitor), mass of the extended envelope, velocity of the radiative shock moving outwards, and the predicted time of explosion are all encoded in the analytic formalism that we build for the primary peak observed in these double peaked light curves.

Over the past decade, four popular analytic formalisms have been presented:
1. PIRO 2015: Model parameters = radius of envelope ($R_e$), mass of envelope ($M_e$), and offset time from explosion ($t_{offset}$)

2. PIRO 2020: Model parameters = radius of envelope ($R_e$), mass of envelope ($M_e$), velocity of shock ($v_e$) and offset time from explosion ($t_{offset}$)

3. Sapir & Waxman 2017 for blue supergiants [n = 3]: Model parameters = radius of envelope ($R_e$), mass of envelope ($M_e$), velocity of shock ($v_e$) and offset time from explosion ($t_{offset}$)

3. Sapir & Waxman 2017 for red supergiants [n = 1.5]: Model parameters = radius of envelope ($R_e$), mass of envelope ($M_e$), velocity of shock ($v_e$) and offset time from explosion ($t_{offset}$)

where n is the density slope parameter.

#### Shock cooling curve is a python package that facilitates using these models to analyze early time data from transients. Currently, two modelling techniques are available:
1. **Non linear least square optimization based curve fitting:** This fits several non linear curves returned by the model by a various combinations of values for the parameters of interest, and returns the parameters that minimize the square error between the non linear curve and observed magnitude data.

2. **Markov Chain Monte Carlo sampling:** This uses maximum likelihood estimation to return the choice of parameters that maxmize the likelihood of observing our data data.

In this problem, we are modelling lightcurves, so our x-data is the time from explosion and the y-data is the photometry in AB Magnitude.

In [ ]:
# this cell ensures that any changes made to the script is reflected / reloaded each time we run a cell.
%load_ext autoreload
%autoreload 2

#### Required Imports

In [ ]:
# The models are stored in shock_cooling_curve.models
from src.shock_cooling_curve.models import PIRO_2015, PIRO_2020, SW_BSG, SW_RSG

# Fitting module
from src.shock_cooling_curve.fitter import Fitter

# Plotting module
from src.shock_cooling_curve.plotter import Plotter

import matplotlib.pyplot as plt

import pandas as pd


Initialize a model by calling one of the specified imports from `shock_cooling_curve.models`. This object inherits from `Supernova`, and will load in all required functionality to build an object that can parse through the provided photometry data and object configuration details.

Note: calling `func?` will display the required inputs and documentation for `func`.

The first argument to initializing a `Supernova` object is `config_file` and is just the name of your configuration file. The supplied configuration file (in this example below it is called `config.ini` - note that only .ini files are accepted) should contain the following information:

1. **start_SN** = start time of the SN (MJD)
2. **end_SC** = end time of shock-cooling curve (MJD)
3. **ebv_host** = extinction of host
4. **ebv_mw** =  extinction of MW

5. **mcore** = mass of core (solar masses)
6. **bestEk3** = kinetic energy of ejecta (10^51 erg/s)

7. **dist_SN** = distance to SN (Mpc)

8. **FILENAME** =  photometry data file (csv)
9. **OBJNAME** = the name of the target object

A template of this the configuration file is available [here](https://github.com/padma18-vb/shock_cooling_curve/tree/main/templates). A template of the photometry data file can also be found in the same folder - the column names have to be the same as specified in the provided file and data for all columns in required.

The second argument `path_storage` specifies the path to where your config file (.ini), photometry data (.csv) and where you want any output files to be stored.

If you don't currently have any data, and would like to try using this package or help develop it, photometry data (and the configuration file) for SN2021gno is available under example_data in `shock_cooling_curve`'s repository [here](https://github.com/padma18-vb/shock_cooling_curve).

In [ ]:
test_model = PIRO_2020.PIRO_2020(config_file='config.ini', path_storage='example_data')

`test_model` will contain all the information to process your photometry data. Under the supplied `path_to_storage`, you will find a new csv file called `reduced_data.csv`, which contains extincted magnitudes and only the data observed the specified time frame (in between `start_SN` and `end_SC`). Apart from this, test_model also has all the filter information for different requirement

In [ ]:
fit_test_model = Fitter(test_model)

In [ ]:
curve_fit_results = fit_test_model.simple_curve_fit()

In [ ]:
curve_fit_results

In [ ]:
fit_test_model.display_curve_fit()

In [ ]:
BSG_low = [0.01, 0.01, 0.01, 0.01]
BSG_high = [2, 2, 4, 0.5]

RSG_low = [0.01, 0.01, 0.01, 0.01]
RSG_high = [1, 2, 3, 0.8]

P15_low = [0.1, 0.01, 0.001]
P15_high = [100, 1, 1]

P20_low = [0.1, 0.01, 0.01, 0.01]
P20_high = [500, 1, 10, 0.2]

In [ ]:
test_chain = fit_test_model.MCMC_fit(prior_low = P20_low, prior_high = P20_high, 
                                     nwalkers = 30, nsteps = 500, sigma = 2, minimize_param_space=True)

In [ ]:
import os

In [ ]:
plot_test_model = Plotter(test_model)



In [ ]:
plot_test_model.MCMC_trace(burnin=100)
plt.suptitle('2021gno; Piro 2015')
#plt.savefig('data/2021gno/P2015_2.png', dpi = 200);

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_test_model.MCMC_corner(burnin=200)
plt.suptitle('2021gno; Piro 2015');
#plt.savefig('data/2021gno/P2015_2.png', dpi = 200);

In [ ]:
params, errs = fit_test_model.set_MCMC_bounds_errs(sigma=1)


In [ ]:
params, errs

In [ ]:
fig = plot_test_model.plot_given_parameters(Re = params['Re'], Me = params['Me'], ve=None, of = params['Off'], show=False)
#plt.savefig('data/2021gno/P2015_fit_2.png', dpi = 200);